In [1]:
import pandas as pd
import numpy as np
import re
import warnings

warnings.filterwarnings('ignore')

## docx 파일에서 표 데이터 추출

In [28]:
df = pd.DataFrame(columns=['c1', 'c2', 'c3', 'c4', 'c5'])

In [20]:
from docx import Document

doc = Document('파일경로/파일명.docx')

bool_reg = 0  # register 추출 기준
bool_addr = 0  # address 추출 기준

num_df_row = 0   # 데이터프레임 .loc

for table in doc.tables : 
    num_row = 1  # 테이블마다 행 count
    check_reg = 0  # higher_datas와 lower_datas 저장 기준
    
    for row in table.rows : 
        num_cell = 0
        num_data = 1
        higher_datas = []  # 상위 데이터 리스트
        lower_datas = []  # 하위 데이터 리스트
        
        for cell in row.cells : 
            c = cell.text
                
            if (bool_reg == 1) and (c != 'c_name2') and (num_cell == 0):   # 첫 번째 행 데이터에 도달하면
                reg = c
                num_cell = 1

            elif (bool_addr == 1) and (c != 'c_name3') and (num_cell == 0):  # 두 번째 행 데이터에 도달하면
                higher_datas.append(c)  # c 추가
                higher_datas.append('32.')
                higher_datas.append(reg)  # 위 if문 데이터 추가
                num_cell = 1
                check_reg = 1

            elif (num_row >= 4) and (check_reg== 1) and(num_data == 1) : 
                lower_datas.append(c)  # 하위 데이터에 bit 추가
                num_data += 1
            elif (num_row >= 4) and (check_reg== 1) and (num_data == 2) :
                lower_datas.insert(0,c)  # 하위 데이터에 name 추가
                num_data += 1
            elif (num_row >= 4) and (check_reg== 1) and (num_data == 3) :
                lower_datas.append(c)  # 하위 데이터에 type 추가
                num_data += 1
            elif (num_row >= 4) and (check_reg== 1) and (num_data == 4) :
                num_data += 1  # 4번째 건너뛰기
            elif (num_row >= 4) and (check_reg== 1) and (num_data == 5) :    
                num_data = 1  # 5번째 건너뒤고 num_data 초기화
            ################################################    
            if c == 'c_name1' :
                bool_reg = 1
                check_reg = 0
            elif c == 'c_name2' :
                num_cell = 0 # num_cell 초기화
                bool_reg = 0 # bool_reg 초기화
                bool_addr = 1
            elif c == 'c_name3' : 
                num_cell = 0 # num_cell 초기화
                bool_addr = 0 # bool_addr 초기화
            #############################################
        num_row += 1
        #################################################
        if len(higher_datas) != 0 :
            higher_datas.append('빈 칸')  # 공백 칸에 의미 없는 데이터 추가
            higher_datas.append('빈 칸')  # 공백 칸에 의미 없는 데이터 추가
            
            df.loc[num_df_row] = higher_datas
            
            num_df_row += 1
            
        if len(lower_datas) != 0 :
            lower_datas.insert(0,'빈 칸')  # 공백 칸에 의미 없는 데이터 추가
            lower_datas.insert(0,'빈 칸')  # 공백 칸에 의미 없는 데이터 추가

            df.loc[num_df_row] = lower_datas
            
            num_df_row += 1

## 데이터프레임으로 변환 후 가공 >> csv 저장

### 1. Address 가공

In [21]:
df['c1'] = df['c1'].map(lambda x : x[-2:] if x[-3] == '특정 문자1' else x[-3:])

df['c1'] = df['c1'].map(lambda x : '특정 문자2'+x)

df['c1'] = df['c1'].map(lambda x : '' if x == '특정 문자2빈 칸' else x)

### 2. From / To 가공

In [22]:
df = df.replace({"빈 칸":''})

In [23]:
df['c7'] = df['c4'].map(lambda x : re.findall(r'^[0-9]+', x)[0]+"." if x != '' else x)
df['c6'] = df['c4'].map(lambda x : re.findall(r'[0-9]+$', x)[0]+"." if x != '' else x)

### 3. RW 교체

In [24]:
df['c5'] = df['c5'].replace({r'^W.*':'W로 시작하는 무언가'},regex=True)
df['c5'] = df['c5'].replace({'R로 시작하는 무언가':"R로 시작하는 무언가", "R":"R로 시작하는 무언가"})

### 4. csv 저장

In [25]:
df["c8"] = ''

In [26]:
df = df[["컬럼을 재정렬"]]

In [27]:
df.to_csv('df_test.csv')